In [1]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'nepalitimes_com'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
publish_source = 'nepalitimes.com'
country = 'Nepal'
language = 'English'
images_path = []

    cat_pages = c.download_page('https://www.nepalitimes.com/')

for _ in cat_pages.split('<li itemscope="itemscope"')[1:]:
    
    cat_url = c.scrap('href="(.*?)"', _)
    if '/about-town/' in cat_url:
        continue
    if 'archive/list' in cat_url:
        continue
    if '#' in cat_url:
        continue
    print(cat_url)

    logger.info(f'Fetching cat url {cat_url}\n')
    cat_page = c.download_page(cat_url)
    
    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) # writes error message with error code
        unable_to_fetch_cat_url += 1
        continue

    for i in cat_page.split('<a itemprop="url"')[1:]:

        # source_link
        source_link = c.scrap('href="(.*?)"', i)
        print(source_link)
                            
        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))
        page = c.download_page(source_link)
        if page.startswith('Unable to fetch'):
                logger.info(page) # writes error message with error code
                unable_to_fetch_article_url += 1
                continue 
       
        # source_headline
        source_headline = c.scrap('<meta property="og:title"\s*content="(.*?)"', page)
        
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
         # Date and time
        pub_date, publish_time = '', ''

        try:
            date_time_str = c.scrap('class="dates">.*?>(.*?)<', page) + "00:00:00" # March 16, 2021
            date_time_str = re.sub(" ","",date_time_str,re.S)
            date_time_obj = datetime.strptime(date_time_str, '%B%d,%Y%H:%M:%S')
            ist_date_time = date_time_obj - timedelta(hours = 0,minutes = 0)  # utc+8 to ist
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
        except:
            pass

        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
#         if pub_date != created_on:
#             break    

        logger.info(f'Fetching {source_link}\n')
        # print(f'Fetching {source_link}\n')   

        # source_content          
        source_content = c.scrap('class="elementor-text-editor\s*elementor-clearfix">(.*?) </article>', page)
        source_content = re.sub('Advertisement' , '', source_content, flags=re.S)
        source_content = c.strip_html(source_content)
       
        # skip if content not found
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')
            skipped_due_to_content += 1
            continue

        # journalist
        journalist =  c.scrap(' <meta\s*name="author"\s*content="(.*?)"',page)
        print(journalist)
        if not journalist: journalist = 'NA'

        # current date and time 
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

        cl_data.insert_one(clientdata)  
        no_of_data += 1


logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')
     

https://www.nepalitimes.com/
https://www.nepalitimes.com/nt/latest/
https://www.nepalitimes.com/latest/tackling-tb-amidst-a-pandemic/
Kritika Dixit
https://www.nepalitimes.com/latest/spring-returns-to-nepals-mountaineering/
Tulsi Rauniyar
https://www.nepalitimes.com/latest/nepali-women-survive-pandemic-economy/
Namrata Sharma
https://www.nepalitimes.com/banner/central-nepal-engulfed-in-smoke/
Nepali Times
https://www.nepalitimes.com/latest/nepals-economic-downturn-1-year-after-lockdown/
Ramesh Kumar
https://www.nepalitimes.com/latest/peak-performance/
Tulsi Rauniyar
https://www.nepalitimes.com/banner/too-little-too-much-or-too-polluted/
Karun Dewan
https://www.nepalitimes.com/latest/taxing-process-of-fund-reclaiming-in-nepal/
Ramesh Kumar
https://www.nepalitimes.com/banner/cabinet-indecision-delays-pokhara-airport/
Krishnamani Baral
https://www.nepalitimes.com/latest/police-track-india-nepal-elephant-trafficking/
Alisha Sijapati and Tufan Neupane
https://www.nepalitimes.com/latest/1-6-

Anil Chitrakar
https://www.nepalitimes.com/opinion/work-for-food/
Anil Chitrakar
https://www.nepalitimes.com/opinion/common-sense-is-uncommon-in-fighting-covid-19/
Anil Chitrakar
https://www.nepalitimes.com/nt/opinion/so-far-so-good/
https://www.nepalitimes.com/opinion/the-vagaries-of-fate/
Lisa Choegyal
https://www.nepalitimes.com/opinion/collateral-benefits/
Lisa Choegyal
https://www.nepalitimes.com/opinion/250-years-of-nepals-history-through-art/
Lisa Choegyal
https://www.nepalitimes.com/opinion/a-postcard-from-jan-morris-1926-2020/
Lisa Choegyal
https://www.nepalitimes.com/opinion/once-in-a-halloween-blue-moon-in-kathmandu/
Lisa Choegyal
https://www.nepalitimes.com/opinion/the-himalaya-mourn-a-saintly-leader/
Lisa Choegyal
https://www.nepalitimes.com/banner/the-elephant-dilemma/
Lisa Choegyal
https://www.nepalitimes.com/opinion/father-moran-and-nepals-jesuits/
Lisa Choegyal
https://www.nepalitimes.com/opinion/nepals-tourism-now-has-to-re-start-from-zero/
Lisa Choegyal
https://www.n